## Script for creating training sample
### Johan Spanners images

1) you need to flatten Johans file structure  
2) Do it on his drive in a new folder called; flatten_folder  
3) Rename each image acording to a new ID; JS + N_i  (JS = Johan Spanner)
4) Draw 5000 random images  
5) Save in new folder: sample_5000  
5) crate csv frame - that is csv with column names and row index  
6) column names = features  
7) row index = image ID  
8) codebook  
9) Page in code book showing the order of the column names so they don't have to scroll 

to mount external harddriv:
> sudo mkdir /mnt/f # f is just random letter for the mounting point  
> sudo mount -t drvfs d: /mnt/f # f is the letter choosen above, d is the drive name

Now you can access his files at location: **/mnt/f/'October Johan Archive'/Spanner**

You then have the folders Published, Raw, Submitted.

# Code

In [1]:
import os
import cv2
from iptcinfo3 import IPTCInfo
from PIL import Image
import re
import zipfile
import shutil
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
# external_dir =  "/mnt/f/OctoberJohanArchive"
external_dir =  "/mnt/f/OctoberJohanArchiveTest" # only the published images
flatten_dir =  "/mnt/f/OctoberJohanArchiveFlatten"

In [3]:
def count_files(external_dir):
    
    """Counts files in folder tree. Sanity check, and fits in the function flatten_dirs()"""
    
    count = 0
    
    for root, dirs, files in os.walk(external_dir):
        for name in files:
            count += 1
        
    return(count)

In [5]:
def mk_flatten_dir(flatten_dir):
    
    """Creates a new folder which is to be the location of the flattened folder tree. fits in the function flatten_dirs()"""
    
    if not os.path.exists(flatten_dir):
        os.mkdir(flatten_dir) # makes new dir                     
        
    # Or REPLACE old if exist
    else:
        shutil.rmtree(flatten_dir) # delets dir and content.. FOREVER!
        os.mkdir(flatten_dir) # makes new dir


In [12]:
def flatten_dirs(external_dir, flatten_dir):
    
    time_start = time.time()
    mk_flatten_dir(flatten_dir) # creates the new folder
    files_count = count_files(external_dir) # counts all files to be moved
    counter = 0 # counter for iterations and image ID
    count_move = 0
    count_exceptions = 0 # count files not being move, due to them not being images.
    ID_front = "JS" # Johan Spanner
    
    print(f'\nstarts moving files from: {external_dir}\nto new shared parent dir: {flatten_dir}...\n')
    
    for root, dirs, files in os.walk(external_dir):
        
        for name in files: 
            
            counter += 1
            print(f'Handling file {counter} of {files_count}', end= '\r')
                        
            old_path = os.path.join(root, name) #all the info for the IPTC should be in in old_path name
            
            try: # you only wnat to move actual images
                
                # verify that is an image
                img = Image.open(old_path)  # open the image file
                img.verify()  # verify that it is a good image, without decoding it.. quite fast
                img.close()
                
                # new path
                count_move += 1
                new_path = flatten_dir + "/" + ID_front + str(count_move) + ".jpeg" # path and image ID - could also put the ID in the IPTC now you are at it.
                
                # Moving day..
                shutil.copyfile(old_path, new_path) # x6 slower then move.. Do IPTC move to
#                 shutil.move(old_path, new_path)# will delete the original path. Do IPTC also move? 
#                 shutil.copy(old_path, new_path) # error for both copy and copy2

            except Exception as e: # Non images you just pass
        
                count_exceptions += 1
                pass
                
    # Logging time it takes to complete operation..
    time_end = time.time()
    time_run = round((time_end - time_start)/60,2)
                    
    print(f'\n\nDone moving files to parent dir: {flatten_dir}...')

    print(f'File(s) moved: {counter-count_exceptions} out of {files_count} files')
    print(f'File(s) not moved (not images): {count_exceptions} out of {files_count} files')
    
    print(f'\nALL DONE!\nThe operation took {time_run} minutes')
    


In [ ]:
flatten_dirs(external_dir, flatten_dir)


starts moving files from: /mnt/f/OctoberJohanArchiveTest
to new shared parent dir: /mnt/f/OctoberJohanArchiveFlatten...

